### This notebook will be used for the Capstone Project!

In [0]:
import pandas as pd
import numpy as np

In [23]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


In [0]:
fs_client_id = 'Y3SRQ4RDK2T4EWBAHCQ1UHZPVP1CMCJZF2WGS0IO1AWTEE1R'
fs_client_secret = 'ME101ALETLMOR4OSUNHLGSYG3XKZBT04OK4TWK3YIUDQWSMS'

### Problem description and background

### Importing Libraries

In [0]:
import json
from bs4 import BeautifulSoup
import requests
from folium import Map, CircleMarker, Popup
from folium.plugins import HeatMap
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

#### Loading dataset from NYPD Public Datasets (Complaint Data 2019)

In [4]:
columns = ['CMPLNT_FR_DT', 'CMPLNT_FR_TM', 'Latitude', 'Longitude']

raw_nypd_df = pd.read_csv('https://raw.githubusercontent.com/devrrad/Coursera_Capstone/master/NYPD_Complaint_Data_2019_Manhattan.csv', usecols=columns)
nypd_df = raw_nypd_df
nypd_df.head(10)

,CMPLNT_FR_DT,CMPLNT_FR_TM,Latitude,Longitude
0,05/15/2019,01:00:00,40.813775,-73.940054
1,04/09/2019,19:23:00,40.737369,-73.996840
2,06/24/2019,09:32:00,40.808374,-73.946886
3,01/03/2019,09:00:00,40.762341,-73.964549
4,06/22/2019,17:40:00,40.735435,-73.979477
5,06/29/2019,03:50:00,40.713719,-73.986088
6,06/26/2019,14:00:00,40.799086,-73.947619
7,06/20/2019,10:00:00,40.777897,-73.984017
8,06/30/2019,00:25:00,40.797060,-73.960753
9,06/28/2019,09:30:00,40.713588,-73.995943


Creating a new datetime column to manage the time component of each event

In [5]:
nypd_df['Datetime'] = nypd_df['CMPLNT_FR_DT'] + ' ' + nypd_df['CMPLNT_FR_TM']
nypd_df['Datetime'] = pd.to_datetime(nypd_df['Datetime'])
nypd_df.drop(columns=['CMPLNT_FR_DT', 'CMPLNT_FR_TM'], inplace=True)
nypd_df.dtypes

Latitude            float64
Longitude           float64
Datetime     datetime64[ns]
dtype: object

In [6]:
nypd_df.head()

,Latitude,Longitude,Datetime
0,40.813775,-73.940054,2019-05-15 01:00:00
1,40.737369,-73.996840,2019-04-09 19:23:00
2,40.808374,-73.946886,2019-06-24 09:32:00
3,40.762341,-73.964549,2019-01-03 09:00:00
4,40.735435,-73.979477,2019-06-22 17:40:00


Sorting by newest events to determine which are the most recent complaints made available

In [7]:
nypd_df.sort_values('Datetime', ascending=False, inplace=True)
nypd_df.head()

,Latitude,Longitude,Datetime
16224,40.773218,-73.988466,2019-06-30 23:18:00
16830,40.788721,-73.939997,2019-06-30 23:15:00
13877,40.757916,-73.989299,2019-06-30 23:15:00
10894,40.763438,-73.992702,2019-06-30 22:50:00
6331,40.786182,-73.945697,2019-06-30 22:43:00


Dropping unnecesary columns

In [8]:
nypd_df.set_index(['Datetime'], drop=True, inplace=True)
nypd_df.head()

,Latitude,Longitude
Datetime,,
2019-06-30 23:18:00,40.773218,-73.988466
2019-06-30 23:15:00,40.788721,-73.939997
2019-06-30 23:15:00,40.757916,-73.989299
2019-06-30 22:50:00,40.763438,-73.992702
2019-06-30 22:43:00,40.786182,-73.945697


Filtering to a month between 01/06/2019 and 2019/06/30 to reduce the dataset size to analyze

In [9]:
nypd_df = nypd_df.loc['2019-06-30':'2019-06-01']
nypd_df.shape

(8794, 2)

In [10]:
nypd_df.reset_index(inplace=True)
nypd_df.head()

,Datetime,Latitude,Longitude
0,2019-06-30 00:00:00,40.733883,-74.005428
1,2019-06-29 23:53:00,40.829610,-73.946208
2,2019-06-29 23:50:00,40.836617,-73.941226
3,2019-06-29 23:50:00,40.793627,-73.972406
4,2019-06-29 23:32:00,40.744362,-74.006598


Splitting the dataset by time windows to see if the hour of the complaints affect the location and quantity the events

In [61]:
time_window_1 = { 'start': 0, 'end': 7}
time_window_2 = { 'start': 8, 'end': 15}
time_window_3 = { 'start': 16, 'end': 23}

nypd_df['tw1'] = ((nypd_df['Datetime'].dt.hour >= time_window_1['start']) & (nypd_df['Datetime'].dt.hour <= time_window_1['end']))
nypd_df['tw2'] = ((nypd_df['Datetime'].dt.hour >= time_window_2['start']) & (nypd_df['Datetime'].dt.hour <= time_window_2['end']))
nypd_df['tw3'] = ((nypd_df['Datetime'].dt.hour >= time_window_3['start']) & (nypd_df['Datetime'].dt.hour <= time_window_3['end']))
nypd_df.describe(include='all')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-vie

,Label,Datetime,Latitude,Longitude,tw1,tw2,tw3
count,8794.000000,8794,8794.000000,8794.000000,8794,8794,8794
unique,NaN,5146,NaN,NaN,2,2,2
top,NaN,2019-06-11 00:01:00,NaN,NaN,False,False,False
freq,NaN,18,NaN,NaN,7088,5325,5175
first,NaN,2019-06-02 00:00:00,NaN,NaN,NaN,NaN,NaN
last,NaN,2019-06-30 00:00:00,NaN,NaN,NaN,NaN,NaN
mean,4.829088,NaN,40.772918,-73.969790,NaN,NaN,NaN
std,2.739871,NaN,0.040791,0.024697,NaN,NaN,NaN
min,0.000000,NaN,40.701489,-74.017790,NaN,NaN,NaN
25%,3.000000,NaN,40.740160,-73.989807,NaN,NaN,NaN


Ploting events between the 00:00 and 07:59

In [12]:
map = Map(location=[40.783060, -73.971249], zoom_start=12)
HeatMap(data=nypd_df[nypd_df['tw1'] == True][['Latitude', 'Longitude']].head(100).groupby(['Latitude', 'Longitude']).sum().reset_index().values.tolist()).add_to(map)
map

Ploting events between the 08:00 and 15:59

In [13]:
map = Map(location=[40.783060, -73.971249], zoom_start=12)
HeatMap(data=nypd_df[nypd_df['tw2'] == True][['Latitude', 'Longitude']].head(100).groupby(['Latitude', 'Longitude']).sum().reset_index().values.tolist()).add_to(map)
map

Ploting events between the 16:00 and 23:59

In [14]:
map = Map(location=[40.783060, -73.971249], zoom_start=12)
HeatMap(data=nypd_df[nypd_df['tw3'] == True][['Latitude', 'Longitude']].head(100).groupby(['Latitude', 'Longitude']).sum().reset_index().values.tolist()).add_to(map)
map

#### Note:

From the heatmaps plotting we can see a variation in the location of the hot spot of generated complaints corresponding to the highest number of events

#### Generating clusters of events by geospatial info

In [15]:
n_clusters = 10
kmeans = KMeans(n_clusters=n_clusters)
kmeans.fit(nypd_df[['Latitude', 'Longitude']])
kmeans.labels_

array([9, 1, 1, ..., 2, 9, 8], dtype=int32)

Adding the clusters labels to dataframe

In [0]:
nypd_df.insert(loc=0, column='Label', value=kmeans.labels_)

In [17]:
nypd_df.head()

,Label,Datetime,Latitude,Longitude,tw1,tw2,tw3
0,9,2019-06-30 00:00:00,40.733883,-74.005428,True,False,False
1,1,2019-06-29 23:53:00,40.829610,-73.946208,False,False,True
2,1,2019-06-29 23:50:00,40.836617,-73.941226,False,False,True
3,7,2019-06-29 23:50:00,40.793627,-73.972406,False,False,True
4,9,2019-06-29 23:32:00,40.744362,-74.006598,False,False,True


#### Plotting the generated clusters

In [18]:
map_clusters = Map(location=[40.783060, -73.971249], zoom_start=12)

x = np.arange(n_clusters)
ys = [i + x + (i*x)**2 for i in range(n_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

for lat, lon, cluster in zip(nypd_df['Latitude'].head(500), nypd_df['Longitude'].head(500), nypd_df['Label'].head(500)):
    CircleMarker(
        [lat, lon],
        radius=5,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Creating a new dataframe of clusters and calculating the incidents rate by cluster

The latitude and longitude of the cluster correspond to the center of each cluster

In [19]:
df = pd.DataFrame()
lats = []
lngs = []
clusters = []
incidents_count = []

for cluster in nypd_df['Label'].unique():
  lats.append(nypd_df[nypd_df['Label'] == cluster]['Latitude'].mean())
  lngs.append(nypd_df[nypd_df['Label'] == cluster]['Longitude'].mean())
  incidents_count.append(len(nypd_df[nypd_df['Label'] == cluster].values)/len(nypd_df))
  clusters.append(cluster)

df.insert(loc=0,column='Latitude',value=lats)
df.insert(loc=0,column='Longitude',value=lngs)
df.insert(loc=0,column='Cluster',value=clusters)
df.insert(loc=0,column='Incidents_Rate', value=incidents_count)

df

,Incidents_Rate,Cluster,Longitude,Latitude
0,0.109279,9,-73.997819,40.739274
1,0.083239,1,-73.942302,40.829724
2,0.070389,7,-73.974556,40.786797
3,0.193768,6,-73.984566,40.756282
4,0.108597,8,-73.951042,40.810280
5,0.102456,4,-73.984331,40.725526
6,0.119627,3,-73.940346,40.795409
7,0.057426,5,-73.929363,40.856558
8,0.091199,2,-74.004067,40.714472
9,0.064021,0,-73.958713,40.770263


#### Plotting each cluster centroid

In [20]:
map_clusters = Map(location=[40.783060, -73.971249], zoom_start=12)

for lat, lon, inc_rate, cluster in zip(df['Latitude'], df['Longitude'], df['Incidents_Rate'], df['Cluster']):
    cluster_label = '{}, {}'.format(cluster, inc_rate)
    label = Popup(cluster_label, parse_html=True)
    
    CircleMarker(
        [lat, lon],
        popup=label,
        radius=5,
        fill=True,
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [0]:
def getNearbyVenues(latitudes, longitudes, radius=500, limit=10):
    venues_list=[]
    for latitude, longitude in zip(latitudes, longitudes):
        params = dict(
          client_id=fs_client_id,
          client_secret=fs_client_secret,
          v='20180323',
          ll='{},{}'.format(latitude, longitude),
          radius=radius,
          limit=limit)

        url = 'https://api.foursquare.com/v2/venues/explore'
        response = requests.get(url, params).json()["response"]['groups'][0]['items']
        venues_list.append([(
            latitude, 
            longitude, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in response])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Latitude', 
                  'Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Getting venues nearby to the cluster centroid

In [32]:
venues_df = getNearbyVenues(latitudes=df['Latitude'], longitudes=df['Longitude'])
venues_df.head()

,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,40.739274,-73.997819,Rubin Museum of Art,40.740110,-73.997807,Art Museum
1,40.739274,-73.997819,Bar B,40.739471,-73.999074,Tapas Restaurant
2,40.739274,-73.997819,Coppelia,40.738819,-73.999908,Cuban Restaurant
3,40.739274,-73.997819,Yanni’s Coffee,40.739768,-73.998884,Coffee Shop
4,40.739274,-73.997819,da Umberto,40.739473,-73.995864,Italian Restaurant


Trying to determine if an specific type of venue nearby the location of the incidents is more relevant than other in the ocurrence of complaints

In [57]:
venues_by_cat = venues_df.groupby(['Venue Category'], as_index=False)['Venue'].count().sort_values('Venue', ascending=False)
venues_by_cat.head(10)

,Venue Category,Venue
32,Italian Restaurant,5
28,Gym,5
53,Theater,5
43,Pizza Place,5
14,Coffee Shop,5
17,Cuban Restaurant,3
18,Dance Studio,3
5,Bakery,3
6,Bar,3
50,Taco Place,3


Given the large amount of diverse venues types, i cannot determine the implication of venue categories and the rise of the number of criminal complaints

In [60]:
venues_by_cat['Venue Category'].nunique()

58